# Point Frontiers

### Objective In Words

Decide weights for each of the 10 columns so that the weighted score for each point is maximized subject to constraints that the score for that point is larger than any other point.

Determine which of the rows scores is maximized subject to constraints such that the point is on the frontier curve.

### Data Definitions

$$ \text{Let } A = \{0,1,2,...,9\} \tag{set of columns}$$

$$ P = \{0,1,2,...,4999\} \tag{set of rows} $$

$$ mydata_{i,j} = \text{value for column i for row j} : j \in A, i \in P \tag{values of A in P}$$



### Decisions

$$ \text{Let } x_j = \text{weight for column } j : j \in A \tag{weights for columns} $$

### Objective

$$ max \sum_{j \in A}mydata_{i,j}*x_j, i \in P, j \in A \tag{Weighted Row Total}$$

$$ \text{S.T. } mydata_i, i \in P $$




In [11]:
import pandas as pd

In [12]:
point_df = pd.read_csv("10by5000(1).txt", sep = " ",header=None)

In [13]:
A = list(point_df)
P = list(point_df.index)

In [14]:
mydata = point_df.stack().to_dict()
print(mydata)

, (4900, 4): 277.7411, (4900, 5): 380.3395, (4900, 6): 108.8609, (4900, 7): 192.4448, (4900, 8): 151.8581, (4900, 9): 20.7394, (4901, 0): 3.7353, (4901, 1): 0.0328, (4901, 2): 2.3133, (4901, 3): 0.8674, (4901, 4): 8.7675, (4901, 5): 1.5269, (4901, 6): 5.8746, (4901, 7): 1.9779, (4901, 8): 2.5974, (4901, 9): 0.9794, (4902, 0): 49.7762, (4902, 1): 1.0003, (4902, 2): 295.1515, (4902, 3): 38.629, (4902, 4): 199.1373, (4902, 5): 33.2137, (4902, 6): 140.6576, (4902, 7): 105.4115, (4902, 8): 19.875999999999998, (4902, 9): 17.7449, (4903, 0): 161.4475, (4903, 1): 0.9531, (4903, 2): 97.9562, (4903, 3): 0.8237, (4903, 4): 268.5113, (4903, 5): 114.3979, (4903, 6): 34.0388, (4903, 7): 158.532, (4903, 8): 12.1243, (4903, 9): 48.0033, (4904, 0): 125.6936, (4904, 1): 3.2276, (4904, 2): 137.5802, (4904, 3): 47.9578, (4904, 4): 50.7059, (4904, 5): 48.5454, (4904, 6): 47.7594, (4904, 7): 59.4195, (4904, 8): 35.6424, (4904, 9): 8.3284, (4905, 0): 152.2539, (4905, 1): 0.1982, (4905, 2): 230.726, (4905, 3)

In [15]:
from pyomo.environ import *

model = ConcreteModel()

In [16]:
try: 
    model.del_component(model.x)
    model.del_component(model.x_index)
except:
    print("Defining variables for first time")
model.x = Var(A, domain=NonNegativeReals)


Defining variables for first time


In [17]:
try:
    model.del_component(model.currPoint)
    model.del_component(model.currPoint_index)
except:
    print("Defining currPoint for the first time")
model.currPoint = Param(A, mutable=True)
model.pprint()

Defining currPoint for the first time
2 Set Declarations
    currPoint_index : Dim=0, Dimen=1, Size=10, Domain=None, Ordered=False, Bounds=(0, 9)
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    x_index : Dim=0, Dimen=1, Size=10, Domain=None, Ordered=False, Bounds=(0, 9)
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

1 Param Declarations
    currPoint : Size=0, Index=currPoint_index, Domain=Any, Default=None, Mutable=True
        Key : Value

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : False :  True : NonNegativeReals
          6 :     0 :  None :

In [18]:
try:
    model.del_component(model.TotalScore)
except:
    print("Defining objective for the first time")
model.TotalScore = Objective(
    expr = sum(model.currPoint[j]*model.x[j] for j in A), sense=maximize)

Defining objective for the first time


In [19]:
try:
    model.del_component(model.ScoreComparison)
    model.del_component(model.ScoreComparison_index)
except:
    print("Defining ScoreComparison for the first time")
model.ScoreComparison = ConstraintList()
for i in P:
    model.ScoreComparison.add(
        sum(mydata[i,j]*model.x[j] for j in A) <=  sum(model.currPoint[j]*model.x[j] for j in A))

Defining ScoreComparison for the first time


In [20]:
try:
    model.del_component(model.Normalization)
    model.del_component(model.Normalization_index)
except:
    print("Defining Normalization for the first time")
model.Normalization = Constraint(
    expr = sum(model.x[j] for j in A) <= 1)
model.pprint()

1851*x[2] + 19.8352*x[3] + 26.0375*x[4] + 66.8546*x[5] + 187.2164*x[6] + 12.4592*x[7] + 33.2214*x[8] + 5.3336*x[9] - (currPoint[0]*x[0] + currPoint[1]*x[1] + currPoint[2]*x[2] + currPoint[3]*x[3] + currPoint[4]*x[4] + currPoint[5]*x[5] + currPoint[6]*x[6] + currPoint[7]*x[7] + currPoint[8]*x[8] + currPoint[9]*x[9]) :   0.0 :   True
        4955 :  -Inf :                                  58.3669*x[0] + 0.5159*x[1] + 211.8566*x[2] + 12.3462*x[3] + 163.5023*x[4] + 69.183*x[5] + 327.2617*x[6] + 45.5924*x[7] + 125.7948*x[8] + 25.1984*x[9] - (currPoint[0]*x[0] + currPoint[1]*x[1] + currPoint[2]*x[2] + currPoint[3]*x[3] + currPoint[4]*x[4] + currPoint[5]*x[5] + currPoint[6]*x[6] + currPoint[7]*x[7] + currPoint[8]*x[8] + currPoint[9]*x[9]) :   0.0 :   True
        4956 :  -Inf :                                90.6634*x[0] + 4.3831*x[1] + 316.6464*x[2] + 126.9478*x[3] + 338.2109*x[4] + 62.3227*x[5] + 112.9649*x[6] + 274.6635*x[7] + 162.0972*x[8] + 6.2751*x[9] - (currPoint[0]*x[0] + currPoint[1]

In [14]:
isFrontierPoint = []


In [17]:
solver = SolverFactory('glpk')
for k in P:
    for j in A:
        model.currPoint[j] = mydata[k,j]
    print(k)
    status = solver.solve(model)
    if value(model.TotalScore) > 0.0:
        print (k, value(model.TotalScore))
        isFrontierPoint.append(k)

2
3163
3164
3165
3165 271.45063425623516
3166
3167
3168
3168 201.82782864550717
3169
3170
3171
3172
3173
3174
3175
3176
3176 78.48306370228131
3177
3178
3178 454.5494460721387
3179
3179 380.4190087267296
3180
3180 189.15205826428092
3181
3182
3183
3183 401.715227762194
3184
3185
3186
3187
3188
3189
3190
3190 21.614633234983014
3191
3192
3193
3193 77.5224644886259
3194
3195
3195 127.28321365289264
3196
3196 184.19490663715172
3197
3198
3199
3200
3201
3201 420.32407654213927
3202
3203
3204
3205
3205 596.4653844005248
3206
3207
3207 255.0103315331749
3208
3208 157.16961618582388
3209
3210
3210 215.12511783754806
3211
3211 69.3696068388422
3212
3212 268.55012661913923
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3222 196.2495745974163
3223
3224
3225
3225 172.74916211620854
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3238 129.1480493149988
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3250 77.73113612164875
3251
3251 188.21868966014762
3252
3253
3254


In [18]:
print(isFrontierPoint)

[6, 8, 16, 25, 26, 28, 39, 45, 48, 53, 60, 61, 69, 77, 78, 82, 83, 84, 88, 91, 92, 93, 98, 107, 108, 110, 113, 117, 119, 120, 121, 128, 132, 135, 137, 139, 140, 144, 163, 173, 178, 180, 182, 183, 184, 191, 198, 202, 207, 208, 209, 213, 218, 221, 232, 233, 236, 238, 241, 242, 249, 250, 252, 268, 269, 270, 271, 274, 282, 284, 287, 292, 295, 296, 312, 313, 320, 321, 324, 326, 334, 342, 348, 354, 360, 364, 369, 370, 371, 372, 376, 380, 387, 398, 399, 407, 410, 411, 418, 426, 428, 429, 441, 442, 446, 457, 465, 469, 472, 474, 477, 485, 490, 495, 496, 499, 502, 505, 506, 507, 511, 521, 524, 528, 532, 536, 541, 543, 545, 551, 554, 555, 558, 560, 565, 568, 569, 570, 573, 576, 577, 579, 587, 588, 589, 592, 593, 594, 602, 603, 620, 622, 626, 627, 633, 634, 635, 637, 639, 648, 649, 652, 662, 666, 667, 671, 675, 676, 679, 680, 688, 699, 703, 704, 709, 710, 715, 716, 729, 732, 740, 744, 755, 757, 759, 762, 765, 771, 776, 777, 789, 790, 798, 799, 807, 808, 811, 816, 817, 821, 826, 831, 832, 834, 841,